* https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api
* https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Population_Density#bands
* https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_UNWPP-Adjusted_Population_Density#description

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee

# only necessary on first run or if the token expires
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
from birdcall_distribution import geo
import json
from shapely.geometry import mapping

geometry = geo.get_california_geometry()
grid = geo.generate_grid(geometry, geo.CA_EXTENT, (0.5, 0.5))

## get stats for a single polygon

In [4]:
polygons = [ee.Geometry.Polygon(mapping(p)["coordinates"]) for p in grid.values()]
polygon = polygons[0]

# scale in meters
scale = 1000

### ground elevation

In [5]:
# Import the USGS ground elevation image.
elv = ee.Image("USGS/SRTMGL1_003")

# elevation data
elv.sample(polygon, scale).aggregate_stats("elevation").getInfo()

{'max': 488,
 'mean': 0.29496173469387754,
 'min': 0,
 'sample_sd': 10.152240823903936,
 'sample_var': 103.06799374654167,
 'sum': 925,
 'sum_sq': 323391,
 'total_count': 3136,
 'total_sd': 10.15062203399906,
 'total_var': 103.0351276771072,
 'valid_count': 3136,
 'weight_sum': 3136,
 'weighted_sum': 925}

In [6]:
elv.reduceRegion(ee.Reducer.percentile([5, 50, 95]), polygon, scale).getInfo()

{'elevation_p5': 0, 'elevation_p50': 0, 'elevation_p95': 0}

### modis land surface temperature

In [7]:
# Initial date of interest (inclusive).
# Final date of interest (exclusive).
i_date = "2017-01-01"
f_date = "2020-01-01"

# Import the MODIS land surface temperature collection.
# temperature day and night, with quality control bands
lst = (
    ee.ImageCollection("MODIS/006/MOD11A1")
    .select("LST_Day_1km", "LST_Night_1km", "QC_Day", "QC_Night")
    .filterDate(i_date, f_date)
)

# LST collection (land surface temperature)
for property in ["LST_Day_1km", "LST_Night_1km"]:
    temp = lst.mean().sample(polygon, scale).aggregate_mean(property).getInfo()
    print(property, temp)

LST_Day_1km 14455.84179004736
LST_Night_1km 14215.800091424046


In [8]:
# get percentiles
lst.mean().reduceRegion(ee.Reducer.percentile([5, 50, 95]), polygon, scale).getInfo()

{'LST_Day_1km_p5': 14394.446153846155,
 'LST_Day_1km_p50': 14454.060240963856,
 'LST_Day_1km_p95': 14519.018975332068,
 'LST_Night_1km_p5': 14203.2578125,
 'LST_Night_1km_p50': 14213.458515283843,
 'LST_Night_1km_p95': 14235.069868995633,
 'QC_Day_p5': 3,
 'QC_Day_p50': 3,
 'QC_Day_p95': 3,
 'QC_Night_p5': 3,
 'QC_Night_p50': 3,
 'QC_Night_p95': 3}

### modis landcover

In [9]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection("MODIS/006/MCD12Q1")

lc_point = lc.first().sample(polygon, scale).first().getInfo()
lc_point

{'type': 'Feature',
 'geometry': None,
 'id': '0',
 'properties': {'LC_Prop1': 3,
  'LC_Prop1_Assessment': 99,
  'LC_Prop2': 3,
  'LC_Prop2_Assessment': 99,
  'LC_Prop3': 3,
  'LC_Prop3_Assessment': 99,
  'LC_Type1': 17,
  'LC_Type2': 0,
  'LC_Type3': 0,
  'LC_Type4': 0,
  'LC_Type5': 0,
  'LW': 1,
  'QC': 2}}

In [10]:
lc.select("LC_Type1").first().sample(polygon, scale).aggregate_histogram(
    "LC_Type1"
).getInfo()

{'1': 1, '17': 2348}

### population density

In [11]:
gpw = ee.ImageCollection("CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density")

gpw.limit(1, "system:time_start", False).first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'unwpp-adjusted_population_density',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [43200, 21600],
   'crs': 'EPSG:4326',
   'crs_transform': [0.00833333333333387,
    0,
    -180,
    0,
    -0.00833333333333387,
    90.00000000001157]}],
 'id': 'CIESIN/GPWv411/GPW_UNWPP-Adjusted_Population_Density/gpw_v4_population_density_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec',
 'version': 1564785062864242,
 'properties': {'Max_Scale': 0,
  'HasZ': 'false',
  'Description': 'The GPWv4 UN WPP-Adjusted Population Density, v4.11_2020 map layer is part of the Gridded Population of the World, Version 4 (GPWv4): Population Density Adjusted to Match 2015 Revision of UN WPP (United Nations World Population Prospects) Country Totals, Revision 11 data set. It displays human population estimates represented by number of persons per square kilometer in each grid cell for the year 2020, derived by dividing the UN adjusted popul

In [12]:
(
    gpw.select("unwpp-adjusted_population_density")
    .limit(1, "system:time_start", False)
    .first()
    .reduceRegion(ee.Reducer.max(), polygons[40], scale)
    .getInfo()
)

{'unwpp-adjusted_population_density': 2438.476318359375}

In [13]:
(
    gpw.select("unwpp-adjusted_population_density")
    .limit(1, "system:time_start", False)
    .first()
    .reduceRegion(ee.Reducer.sum(), polygons[40], scale)
    .getInfo()
)

{'unwpp-adjusted_population_density': 112557.76442598837}

### alternative

In [14]:
gpw = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Density")
(
    gpw.select("population_density")
    .limit(1, "system:time_start", False)
    .first()
    .reduceRegion(ee.Reducer.max(), polygons[40], scale)
    .getInfo()
)

{'population_density': 2511.423828125}

In [15]:
(
    gpw.select("population_density")
    .limit(1, "system:time_start", False)
    .first()
    .reduceRegion(ee.Reducer.sum(), polygons[40], scale)
    .getInfo()
)

{'population_density': 115924.94831829822}